In [1]:
import numpy as np
import pandas as pd
import datetime
import copy
from keras.models import Sequential
from keras.layers import LSTM, Dense
import tensorflow as tf
from tensorflow import keras
from keras.layers import TimeDistributed, Conv1D, MaxPool1D, Flatten, LSTM, Dense, AveragePooling1D, Dropout, Conv2D, MaxPool2D
from keras.models import Sequential
from keras.regularizers import l2
import sklearn.metrics as metrics
from pyarrow import feather

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
NUM_EPOCHS = 100
BATCH_SIZE = 32
client_lr = 3e-4
NUM_PACKET = 5

In [4]:
def data_processing(df, NUM_FEATURES):
   y_train = df['Label']
   flow_id = df['flow_id']

   df = df/255

   X_train = df.drop(['Label', 'flow_id'], axis=1)
   X_train = X_train.to_numpy()

   X_train = X_train.reshape(-1, NUM_PACKET, NUM_FEATURES, 1)
   y_train = y_train.to_numpy()

   y_train = y_train.reshape(-1,NUM_PACKET)[:,-1]
   return X_train, y_train
def DoiNhan(label: np.array):
  for i in range(len(label)):
    if label[i] == 14 or label[i] == 16 or label[i] == 13:
      label[i] = 2
    elif label[i] == 11 or label[i] == 5 or label[i] == 10:
      label[i] = 1
    elif label[i] == 4 or label[i] == 8 or label[i] == 9 or label[i] == 1 or label[i] == 7:
      label[i] = 0

In [5]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [9]:
def train_CNN(X, y, X_test, y_test):
    model = Sequential()
    model.add(Conv2D(filters=128, kernel_size=(5, 5), padding='Same',
                     activation='relu', input_shape=(NUM_PACKET,128,1)))
    model.add(Conv2D(filters=64 , kernel_size=(5, 5), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='Same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='Same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='Same'))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(Conv2D(filters=8, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='Same'))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.1))
    model.add(Dense(3, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=client_lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

    model.fit(X, y, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, shuffle=True)
    predictions = model.predict(X_test, verbose=2, batch_size=BATCH_SIZE)
    y_pred = np.argmax(predictions, axis=-1)

    print(metrics.classification_report(y_test, y_pred))

    return model

In [10]:
import time

def Training_Step():
  path = '/content/drive/MyDrive/Fixed_data/'
  test_name = 'd1test'
  train_name ='d1train'
  BATCH_SIZE = 32

  res = []
  train_data = feather.read_feather(path + train_name + str(1))
  test_data = feather.read_feather(path + test_name + str(1))

  X_train, y_train = data_processing(train_data, 128)
  X_test, y_test = data_processing(test_data, 128)

  DoiNhan(y_train)
  DoiNhan(y_test)

  CNN_model = train_CNN(X_train, y_train, X_test, y_test)
  flows = set(test_data['flow_id'])
  print(flows)
  l = []
  for id in flows:
    df = test_data.loc[test_data['flow_id'] == id]
    X_check, y_check = data_processing(df, 128)
    DoiNhan(y_check)

    start = time.time()
    y_pred = CNN_model.predict(X_check, verbose=0, batch_size=BATCH_SIZE)
    end = time.time()

    length = end - start
    l.append(length)
  res = sum(l) / len(flows)
  return res

In [ ]:
afct = Training_Step()
print(afct)

Epoch 1/100
11/11 [==============================] - 9s 624ms/step - loss: 1.0951 - sparse_categorical_accuracy: 0.5284
Epoch 2/100
 4/11 [=========>....................] - ETA: 7s - loss: 1.0833 - sparse_categorical_accuracy: 0.5469

In [ ]:
print(afct)